In [ ]:
# 1. 安裝函式庫
!pip install git+https://github.com/[your-username]/ImageAlchemy.git

In [ ]:
# 2. 匯入與設定
from image_alchemy import ImageAlchemy
from image_alchemy.utils.visualization import compare_images
from image_alchemy.utils.image_utils import combine_image_and_mask
from image_alchemy.core.pipelines import run_sam_segmentation
from PIL import Image
import requests
from io import BytesIO
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'使用裝置: {device}')
engine = ImageAlchemy(device=device)

In [ ]:
# 3. 載入範例圖片
# 我們將使用一張帶有清晰物件的圖片，以便進行操作。
url = 'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/cat-dog.png'
response = requests.get(url)
original_image = Image.open(BytesIO(response.content)).convert("RGB")

print("原始圖片已載入。")
display(original_image)

In [ ]:
# 首先，定義小狗周圍的邊界框
dog_bbox = [350, 20, 650, 350]

# (可選步驟) 視覺化 SAM 從邊界框生成的精確遮罩
print("正在使用 SAM 從邊界框生成精確遮罩...")
dog_mask = run_sam_segmentation(engine.model_loader, original_image, input_box=dog_bbox)
print("視覺化生成的遮罩疊加在原始圖片上:")
display(combine_image_and_mask(original_image, dog_mask))

In [ ]:
# 現在，直接調用函式庫的主要功能來移除物件
print("\n正在使用 Stable Diffusion Inpainting 移除物件...")
removed_image = engine.manipulation.remove_object(
    image=original_image,
    mask=dog_bbox, # 我們可以直接傳遞邊界框
    prompt="a cat sitting on a rug, photorealistic, high quality"
)

compare_images(original_image, removed_image, after_text="移除小狗後")
print("物件移除完成。")

In [ ]:
# 定義貓的邊界框
cat_bbox = [20, 20, 350, 350]

print("正在生成新背景...")
generated_bg_image = engine.generative.generate_background(
    image=original_image,
    foreground_mask=cat_bbox,
    background_prompt="a cat sitting on a sandy beach, ocean waves in the background, sunny day"
)

compare_images(original_image, generated_bg_image, after_text="新背景：沙灘")

# 儲存並提供下載連結
generated_bg_image.save("final_manipulated_image.jpg")
print("背景生成完成，結果已儲存至 final_manipulated_image.jpg")